# Create SparkSession, import packages

# To Do:
* combine the data with the features file, to get a header
* do variable selection and transformation
* spit out a final `.csv` to use for modeling

# Ideas
* look at the data via kmeans with 6 clusters?
* compute AUC on each variable to see which are important? (after regressing on the var)
* create variables that are combinations of other variables
* look for outliers and then run pca before clustering?
* run multinomial regression with elastic net? (with k-fold cv)
* or maybe try neural nets?
* something time series?


In [6]:
from pyspark.sql import SparkSession, SQLContext
import pyspark.sql.types as typ
import pyspark.sql.functions as F
import os
import pandas as pd
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("xor") \
    .config("spark.executor.memory", '2g') \
    .config('spark.executor.cores', '1') \
    .config('spark.cores.max', '1') \
    .config("spark.driver.memory",'1g') \
    .getOrCreate()

sc = spark.sparkContext
sqlCtx = SQLContext(sc)

In [48]:
#read in with pandas first since the data is weirdly delimited
pandas_train_features = pd.read_csv("Data/X_train.txt", sep='\s+',header=None)
#conversion to spark df
df_train_features = sqlCtx.createDataFrame(pandas_train_features)

In [14]:
#training labels
pandas_train_labels = pd.read_csv("Data/y_train.txt", sep='\s+',header=None)
#conversion to spark df
df_train_labels = sqlCtx.createDataFrame(pandas_train_labels)

In [16]:
#test features
pandas_test_features = pd.read_csv("Data/X_test.txt", sep='\s+',header=None)
#conversion to spark df
df_test_features = sqlCtx.createDataFrame(pandas_test_features)

In [18]:
#test labels
pandas_test_labels = pd.read_csv("Data/y_test.txt", sep='\s+',header=None)
#conversion to spark df
df_test_labels = sqlCtx.createDataFrame(pandas_test_labels)

In [52]:
#header
pandas_header = pd.read_csv("Data/features.txt", sep='\s+',header=None)
#conversion to spark df. this has an index column, so we deselect it
df_header = sqlCtx.createDataFrame(pandas_header)


In [53]:
df_header.show(1)

+---+-----------------+
|  0|                1|
+---+-----------------+
|  1|tBodyAcc-mean()-X|
+---+-----------------+
only showing top 1 row



# Look at class imbalance in our data

+-------+------------------+
|summary|                 0|
+-------+------------------+
|  count|              7352|
|   mean|3.6433623503808485|
| stddev|1.7448018716489957|
|    min|                 1|
|    max|                 6|
+-------+------------------+

